# Vectores de conteo de términos

Cada par consulta-documento $(q,d)$ de los archivos de entrada proporciona información relativa a los diferentes términos de cada consulta en cinco campos diferentes de cada documento, a saber: <b>url</b>, <b>title</b>, <b>headers</b>, <b>body</b> y <b>anchors</b> (el campo <b>pagerank</b> adicional no se utilizará hasta más adelante).

Las funciones de _ranking_ construirán inicialmente los correspondientes cinco vectores de conteo de términos crudos ($rs$, de _raw score_) para cada uno de estos pares $(q,d)$, precisamente a partir de las coincidencias (_hits_) en estos cinco diferentes campos. Estos vectores $rs$ simplemente contarán cuantas veces ocurre cada término de búsqueda en un determinado campo. Para el campo <b>anchor</b>, se asumirá la simplificación de que hay un gran documento que contiene todos los enlaces, con el texto del enlace multiplicado por el campo <b>stanford_anchor_count</b>. Seguiremos un enfoque análogo también para el campo <b>header</b>.
    
Así, para el par $(q,d)$ de ejemplo donde la consulta era $q=[\text{stanford aoerc pool hours}]^T$ y el documento $d=$"http://events.stanford.edu/2014/February/18/" (mostrado en el ejemplo de una celda anterior de este mismo notebook), el vector ${rs}_{b}$ correspondiente al campo <b>body</b> será $[{10 \ 7 \ 1 \ 0}]^T$, dado que había 10 apariciones del término "stanford" en dicho campo, 7 para el término "aoerc", 1 para el término "pool" y ninguna para el término "hours". Análogamente, el vector ${rs}_{a}$ para el campo <b>anchor</b> será simplemente $[\text{0 0 0 0}]^T$, dado que no hay ningún enlace (<b>anchor</b>) para este documento. Finalmente el vector ${rs}_{t}$ para el campo <b>title</b> será $[\text{1 0 0 0}]^T$, $[\text{1 0 0 0}]^T$ también para el vector ${rs}_{u}$ del campo <b>url</b> (estos dos fácilmente deducibles desde los correspondientes campos `title` y `url`, que son únicos en cada documento), y ${rs}_{h}$ $[\text{5 0 0 1}]^T$ para el campo <b>header</b> (este último acumulado para las distintas cabeceras --esto es, "subtítulos"-- presentes en el documento). Todo esto se puede corroborar fácilmente observando el contenido de dicho documento en el archivo de señal original:
    
```   
query: stanford aoerc pool hours
 url: http://events.stanford.edu/2014/February/18/
  title: events at stanford tuesday february 18 2014
  header: stanford university event calendar
  header: teaching sex at stanford
  header: rodin the complete stanford collection
  header: stanford rec trx suspension training
  header: memorial church open visiting hours
  header: alternative transportation counseling tm 3 hour stanford univ shc employees retirees family members
  body_hits: stanford 239 271 318 457 615 642 663 960 966 971
  body_hits: aoerc 349 401 432 530 549 578 596
  body_hits: pool 521
  body_length: 981
  pagerank: 1    
```
    
Un ejemplo adicional, usando la misma consulta, pero un documento de respuesta diferente, para ilustrar los vectores correspondientes a los campos <b>anchor</b>, no presentes en el documento anterior:
```
  url: https://cardinalrec.stanford.edu/facilities/aoerc/
    ...
    anchor_text: gyms aoerc
      stanford_anchor_count: 3
    anchor_text: aoerc
      stanford_anchor_count: 13
    anchor_text: http cardinalrec stanford edu facilities aoerc
      stanford_anchor_count: 4
    anchor_text: arrillaga outdoor education and recreation center aoerc link is external
      stanford_anchor_count: 1
    anchor_text: the arrillaga outdoor education and research center aoerc
      stanford_anchor_count: 2
    anchor_text: aoerc will shutdown for maintenance
      stanford_anchor_count: 2
```

Aquí, el vector para  <b>anchor</b> será $[\text{4 25 0 0}]^T$, dado que sólo hay un total de 4 enlaces (campo <b>stanford_anchor_count</b>) para el término "stanford", pero 25 (=3+13+4+1+2+2) para el término “aoerc”.

(Nótese que, en lo referente al campo  <b>url</b> es necesario _"tokenizar"_ previamente sólo los caracteres alfanuméricos. Nótese también que, al calcular los conteos de términos crudos, todo se hace convirtiendo previamente los _tokens_ a minúsculas).

# Clase base para todos los _scorers_

## Clase _AbstractScorer_

Construimos ahora una clase base abstracta, de la que habrá que ir reimplementando métodos en las clases hijas, conforme vayamos implementando los diferentes métodos de _scoring_. En todo caso, esta clase AbstractScorer recogerá una funcionalidad básica común a todos. Lo fundamental serán los distintos métodos `parse_`_field_, que transforman la información textual disponible en el archivo de señal para cada campo en la correspondiente información numérica. Aprovechamos también aquí para añadir los posibles esquemas de _weighting_ SMART vistos en teoría (si bien en esta práctica no programaremos todas las posibilidades disponibles, sino sólo alguna de las más comúnmente utilizadas):

In [1]:
# Imports necesarios
import sys
import array
import os
import timeit
import contextlib
import math
import urllib.request
import zipfile
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from collections import OrderedDict, Counter, defaultdict
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Definiciones de clases del notebook anterior (P2a)
class Query:
    """Clase utilizada para almacenar una consulta."""
    def __init__(self, query):
        self.query_words = query.split(" ")

    def __iter__(self):
        for w in self.query_words:
            yield w

    def __eq__(self, other):
        if not isinstance(other, Query):
            return False
        return self.query_words == other.query_words

    def __hash__(self):
        return hash(str(self))

    def __str__(self):
        return " ".join(self.query_words)

    __repr__ = __str__

class Document:
    """Clase utilizada para almacenar información útil para un documento."""
    def __init__(self, url):
        self.url = url        # Cadena
        self.title = None     # Cadena
        self.headers = None   # [Lista de cadenas]
        self.body_hits = None # Diccionario: Término->[Lista de posiciones]
        self.body_length = 0  # Entero
        self.pagerank = 0     # Entero
        self.anchors = None   # Diccionario: Cadena->[Conteo total de ocurrencias (anchor_counts)]

    def __iter__(self):
        for u in self.url:
            yield u

    def __str__(self):
        result = [];
        NEW_LINE = "\n"
        result.append("url: "+ self.url + NEW_LINE);
        if (self.title is not None): result.append("title: " + self.title + NEW_LINE);
        if (self.headers is not None): result.append("headers: " + str(self.headers) + NEW_LINE);
        if (self.body_hits is not None): result.append("body_hits: " + str(self.body_hits) + NEW_LINE);
        if (self.body_length != 0): result.append("body_length: " + str(self.body_length) + NEW_LINE);
        if (self.pagerank != 0): result.append("pagerank: " + str(self.pagerank) + NEW_LINE);
        if (self.anchors is not None): result.append("anchors: " + str(self.anchors) + NEW_LINE);
        return " ".join(result)

    __repr__ = __str__

class Idf:
    """Construye un diccionario para poder devolver el IDF de un término."""
    def __init__(self):
        """Construcción del diccionario IDF"""
        try:
            with open("pa3-data/docs.dict", 'rb') as f:
                docs = pkl.load(f)
            self.total_doc_num = len(docs)
            print("Número total de documentos de la colección:", self.total_doc_num)

            with open("pa3-data/terms.dict", 'rb') as f:
                terms = pkl.load(f)
            self.total_term_num = len(terms)
            print("Número total de términos:", self.total_term_num)

            with open('pa3-data/BSBI.dict', 'rb') as f:
                postings_dict, termsID = pkl.load(f)

            self.idf = {}
            self.raw = {}
            for term_id, term_str in enumerate(terms.id_to_str):
                if term_id in postings_dict:
                    num_docs_with_term = postings_dict[term_id][1]
                    self.raw[term_str] = num_docs_with_term
                    self.idf[term_str] = math.log((self.total_doc_num + 1) / (num_docs_with_term + 1))
        except FileNotFoundError:
            print("¡Ficheros de diccionario de documentos / términos / índice no encontrados!")

    def get_raw(self, term):
        return self.raw.get(term, 0)

    def get_idf(self, term):
        if term in self.idf:
            return self.idf[term]
        else:
            return math.log(self.total_doc_num + 1)

def load_train_data(feature_file_name):
    """Carga los datos de entrenamiento."""
    line = None
    url = None
    anchor_text = None
    query = None
    query_dict = {}
    try:
        with open(feature_file_name, 'r', encoding = 'utf8') as f:
            for line in f:
                token_index = line.index(":")
                key = line[:token_index].strip()
                value = line[token_index + 1:].strip()
                if key == "query":
                    query = Query(value)
                    query_dict[query] = {}
                elif key == "url":
                    url = value;
                    query_dict[query][url] = Document(url);
                elif key == "title":
                    query_dict[query][url].title = str(value);
                elif key == "header":
                    if query_dict[query][url].headers is None:
                        query_dict[query][url].headers = []
                    query_dict[query][url].headers.append(value)
                elif key == "body_hits":
                    if query_dict[query][url].body_hits is None:
                        query_dict[query][url].body_hits = {}
                    temp = value.split(" ",maxsplit=1);
                    term = temp[0].strip();
                    if term not in query_dict[query][url].body_hits:
                        positions_int = []
                        query_dict[query][url].body_hits[term] = positions_int
                    else:
                        positions_int = query_dict[query][url].body_hits[term]
                    positions = temp[1].strip().split(" ")
                    for position in positions:
                        positions_int.append(int(position))
                elif key == "body_length":
                    query_dict[query][url].body_length = int(value);
                elif key == "pagerank":
                    query_dict[query][url].pagerank = int(value);
                elif key == "anchor_text":
                    anchor_text = value
                    if query_dict[query][url].anchors is None:
                        query_dict[query][url].anchors = {}
                elif key == "stanford_anchor_count":
                    query_dict[query][url].anchors[anchor_text] = int(value)
    except FileNotFoundError:
        print(f"Fichero {feature_file_name} no encontrado!")
    return query_dict

class IdMap:
    """Clase auxiliar para almacenar mapeos entre strings e identificadores numéricos de tokens."""
    
    def __init__(self):
        """Constructor de la clase IdMap."""
        self.str_to_id = {}  # Diccionario: string -> id numérico
        self.id_to_str = []  # Lista: índice = id, valor = string
        
    def __len__(self):
        """Devuelve el número de elementos en el mapeo."""
        return len(self.id_to_str)
    
    def _get_str(self, i):
        """Devuelve el string correspondiente al id numérico i.
        
        Args:
            i (int): Identificador numérico.
            
        Returns:
            str: String correspondiente al id.
        """
        return self.id_to_str[i]
    
    def _get_id(self, s):
        """Devuelve el id numérico correspondiente al string s.
        Si el string no existe, lo añade y devuelve su nuevo id.
        
        Args:
            s (str): String del que obtener su id.
            
        Returns:
            int: Identificador numérico del string.
        """
        if s not in self.str_to_id:
            # Si no existe, lo añadimos
            new_id = len(self.id_to_str)
            self.str_to_id[s] = new_id
            self.id_to_str.append(s)
            return new_id
        return self.str_to_id[s]
    
    def __getitem__(self, key):
        """Permite acceso mediante corchetes: idmap[key]
        Si key es int, devuelve el string.
        Si key es str, devuelve el id.
        
        Args:
            key: Puede ser int (devuelve string) o str (devuelve id).
            
        Returns:
            El string o id correspondiente.
        """
        if isinstance(key, int):
            return self._get_str(key)
        elif isinstance(key, str):
            return self._get_id(key)
        else:
            raise TypeError("La clave debe ser int o str")
    
    def __contains__(self, key):
        """Permite usar 'in' para verificar existencia.
        
        Args:
            key: Puede ser int o str.
            
        Returns:
            bool: True si existe, False en caso contrario.
        """
        if isinstance(key, int):
            return 0 <= key < len(self.id_to_str)
        elif isinstance(key, str):
            return key in self.str_to_id
        return False
    
    def __repr__(self):
        """Representación en string del objeto."""
        return f"IdMap(size={len(self)})"

# Cargar datos necesarios
print("Cargando datos...")
data_dir = 'pa3-data'
theIDF = Idf()
print()
file_name = os.path.join(data_dir, "pa3.signal.train")
query_dict = load_train_data(file_name)
print(f"Datos cargados: {len(query_dict)} consultas")

Cargando datos...
Número total de documentos de la colección: 98998
Número total de términos: 347071

Datos cargados: 731 consultas


In [2]:
class AbstractScorer:
    """ Una clase básica abstracta para un scorer.
        Implementa una funcionalidad básica de construcción de vectores de consulta y de documento.
        Tendrá que ser extendida adecuadamente por cada scorer específico.
    """
    def __init__(self, idf, query_weight_scheme=None, doc_weight_scheme=None):
        self.idf = idf
        self.TFTYPES = ["url", "title", "body_hits", "header", "anchor"]
        # Esquemas por defecto:
        self.default_query_weight_scheme = {"tf": 'n', "df": 'n', "norm": None} # Esquema natural, no, none
        self.default_doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": None}   # Esquema natural, no, none
        self.query_weight_scheme = query_weight_scheme if query_weight_scheme is not None \
                                   else self.default_query_weight_scheme
        self.doc_weight_scheme = doc_weight_scheme if doc_weight_scheme is not None \
                                 else self.default_doc_weight_scheme

    def parse_url(self, url):
        """Parsea la URL del documento, devolviendo un Counter de los tokens encontrados en la URL.
        Args:
            url: el url del que se va a hacer el parsing.
        Returns:
            Lista de tokens del URL (una vez limpios), y Counter resultado.
        """
        if url:
            url_token_in_term = url.replace("http:",".").replace('/','.').replace('?','.') \
                                   .replace('=','.').replace("%20",".").replace("...",".").replace("..",".")\
                                   .replace('-','.').lower();
            url_token = url_token_in_term.strip(".").split('.')
            return url_token, Counter(url_token)
        else:
            return [], Counter([])

    def parse_title(self, title):
        """Parsea el campo title del documento, devolviendo un Counter de los tokens encontrados en el mismo.
        Args:
            title: el title del que se va a hacer el parsing.
        Returns:
            El Counter resultado.
        """
        if title:
            return Counter(title.split(" "))
        else:
            return Counter([])

    def parse_headers(self, headers):
        """Parsea los campos headers del documento, devolviendo un Counter de los tokens encontrados en los mismos.
        Args:
            headers: la lista de headers sobre los que se va a hacer el parsing.
        Returns:
            El Counter resultado.
        """
        headers_token = []
        # BEGIN YOUR CODE
        if headers is not None:
            for header in headers:
                # Dividir cada header en tokens y añadirlos a la lista
                header_tokens = header.split(" ")
                headers_token.extend(header_tokens)
        # END YOUR CODE
        return Counter(headers_token)

    def parse_anchors(self, anchors):
        """Parsea los campos anchors del documento, devolviendo un Counter de los tokens encontrados en los mismos.
        Args:
            anchors: la lista de anchors sobre los que se va a hacer el parsing.
        Returns:
            El Counter resultado.
        """
        anchor_count_map = Counter({})
        if anchors is not None:
            for anchor in anchors:
                count = anchors[anchor]
                anchor_tokens = anchor.split(" ")
                for anchor_token in anchor_tokens:
                    if(anchor_token in anchor_count_map.keys()):
                        anchor_count_map[anchor_token] += count
                    else:
                        anchor_count_map[anchor_token] = count
        return anchor_count_map

    def parse_body_hits(self, body_hits):
        """Parsea los campos body_hits del documento, devolviendo un Counter de los tokens encontrados en los mismos.
        Args:
            body_hits: la lista de anchors sobre los que se va a hacer el parsing.
        Returns:
            El Counter resultado.
        """
        body_hits_count_map = Counter({})
        #BEGIN YOUR CODE
        if body_hits is not None:
            # body_hits es un diccionario: término -> lista de posiciones
            # El conteo es simplemente la longitud de la lista de posiciones
            for term, positions in body_hits.items():
                body_hits_count_map[term] = len(positions)
        #END YOUR CODE
        return body_hits_count_map

    def get_query_vector(self, q, query_weight_scheme = None):
        """ Obtiene un vector numérico para la consulta q.
        Args:
            q (Query): Query("Una consulta determinada")
        Returns:
            query_vec (dict): El vector resultado.
        """
        # En subclases de esta AbstractScorer, podrían tenerse en cuenta todas las
        # posibilidades SMART, usando diferentes esquemas de frecuencia del término (tf),
        # frecuencia de documento (idf) y normalización. En todo caso, nótese que en
        # general no se suele necesitar normalización para la consulta en ningún caso, ya que
        # dicha normalización no variaría con respecto a todos los documentos resultados de una
        # misma consulta, lo que resultaría en un simple factor de escalado común que no
        # afectaría al posterior ranking de los mismos.
        #
        # if query_weight_scheme is None:
        #     query_weight_scheme = self.query_weight_scheme

        query_vec = {}
        ### BEGIN YOUR CODE (FIXME)
        # En nuestro caso base, usaremos simplemente el contador básico de términos, sin
        # normalización ni uso de idf:
        query_vec = Counter(q.query_words)
        ### END YOUR CODE (FIXME)
        return query_vec

    def get_doc_vector(self, q, d, doc_weight_scheme=None):
        """ Obtiene un vector numérico para el documento d.
        Args:
        q (Query) : Query("Una consulta")
        d (Document) : Query("Una consulta")["Un URL"]
        Returns:
        doc_vec (dict) : Un diccionario de conteo de la frecuencia de términos, con un subdiccionario para
                         cada tipo de campo (tipo_de_campo -> (término -> conteo))
                    Ejemplo: "{'url':   {'stanford': 1, 'aoerc': 0, 'pool': 0, 'hours': 0},
                               'title': {'stanford': 1, 'aoerc': 0, 'pool': 0, 'hours': 0},
                               ...
                               }"
        """
        # De nuevo, podrían considerarse todas las posibilidades SMART en las subclases
        # de esta AbstractScorer, si bien en esta clase base nos contentaremos con un simple
        # conteo crudo de los términos en los distintos campos:
        #
        # if doc_weight_scheme is None:
        #    doc_weight_scheme = self.doc_weight_scheme

        doc_vec = {}
        ### BEGIN YOUR CODE (FIXME)
        # Sólo para depurar:
        # print(f"URL:        {d.url}          ->   {self.parse_url(d.url)}")
        # print(f"TITLE:      {d.title}        ->   {self.parse_title(d.title)}")
        # print(f"HEADERS:    {d.headers}      ->   {self.parse_headers(d.headers)}")
        # print(f"ANCHORS:    {d.anchors}      ->   {self.parse_anchors(d.anchors)}")
        # print(f"BODY_HITS:  {d.body_hits}    ->   {self.parse_body_hits(d.body_hits)}")
        #
        # Simple conteo crudo de los términos por campos:
        _, url_counter = self.parse_url(d.url)
        doc_vec['url'] = url_counter
        doc_vec['title'] = self.parse_title(d.title)
        doc_vec['headers'] = self.parse_headers(d.headers)
        doc_vec['anchors'] = self.parse_anchors(d.anchors)
        doc_vec['body_hits'] = self.parse_body_hits(d.body_hits)
        ### END YOUR CODE (FIXME)
        return doc_vec

    # Métodos no implementados en la clase base; en su caso, serán reimplementados en cada scorer concreto:

    def normalize_doc_vec(self, q, d, doc_vec):
        """ Normalizar el vector de documento.
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
            doc_vec (dict) : El vector de documento
        """
        raise NotImplementedError

    def get_sim_score(self, q, d):
        """ Devuelve la puntuación para una consulta q y documento d dados.
        Args:
            q (Query): la consulta.
            d (Document) : el documento.
        Returns:
            La puntuación para el par (q,d).
        """
        raise NotImplementedError

    def get_net_score(self, q, d):
        """ Calcular el scoring neto entre la consulta y el documento.
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
        Return:
            score (float) : La puntuación resultado.
        """
        raise NotImplementedError

Probamos la clase abstracta con una consulta y un documento:

In [3]:
q = Query("stanford aoerc pool hours")
d = query_dict[q]['http://events.stanford.edu/2014/February/18/']
print("Query q: ", q)
print()
print("Document d: ", d)

a_scorer = AbstractScorer(theIDF)
query_vec = a_scorer.get_query_vector(q)
print(f"Vector consulta:")
print(f"  {query_vec}")
print()
doc_vec = a_scorer.get_doc_vector(q, d)
print(f"Vector documento:")
for k, v in doc_vec.items():
    print(f"  {k:10s} -> {v}")

Query q:  stanford aoerc pool hours

Document d:  url: http://events.stanford.edu/2014/February/18/
 title: events at stanford tuesday february 18 2014
 headers: ['stanford university event calendar', 'teaching sex at stanford', 'rodin the complete stanford collection', 'stanford rec trx suspension training', 'memorial church open visiting hours', 'alternative transportation counseling tm 3 hour stanford univ shc employees retirees family members']
 body_hits: {'stanford': [239, 271, 318, 457, 615, 642, 663, 960, 966, 971], 'aoerc': [349, 401, 432, 530, 549, 578, 596], 'pool': [521]}
 body_length: 981
 pagerank: 1

Vector consulta:
  Counter({'stanford': 1, 'aoerc': 1, 'pool': 1, 'hours': 1})

Vector documento:
  url        -> Counter({'events': 1, 'stanford': 1, 'edu': 1, '2014': 1, 'february': 1, '18': 1})
  title      -> Counter({'events': 1, 'at': 1, 'stanford': 1, 'tuesday': 1, 'february': 1, '18': 1, '2014': 1})
  headers    -> Counter({'stanford': 5, 'university': 1, 'event': 1,

La salida tendría que ser la siguiente:

## Clase _BaselineScorer_

Definimos aquí un simple _"scorer baseline"_, que nos servirá para probar la funcionalidad de la clase abstracta base. La clase `BaselineScorer` heredará directamente de la clase `AbstractScorer`, reimplementando solamente el método `get_sim_score`, que simplemente acumulará, para aquellos términos en la consulta, los contadores absolutos de dichos términos (TFs) en el vector de documento correspondiente, utilizando en cada caso el campo `url`, `title`, `headers`, `anchors`, o `body_hits` que se le indique en cada momento mediante el método `set_field_type(...)`:

In [6]:
class BaselineScorer(AbstractScorer):
    def __init__(self, idf):
        super().__init__(idf)
        self.field_type = "url"  # Campo por defecto de inicialización de la clase.

    def set_field_type(self, field_type):
        self.field_type = field_type

    def get_sim_score(self, q, d):
        score = 0
        # BEGIN YOUR CODE
        # Simplemente acumularemos los TFs de cada término de la query para el documento dado
        
        # Obtener el vector de consulta
        query_vec = self.get_query_vector(q)

        # Obtener el vector del documento (con todos sus campos)
        doc_vec = self.get_doc_vector(q, d)

        # Obtener el Counter del campo específico seleccionado
        field_vec = doc_vec[self.field_type]

        # Para cada término en la consulta, sumar su TF en el campo del documento
        for term in query_vec:
            if term in field_vec:
                score += field_vec[term]
        # END YOUR CODE
        return score

    # En este caso el scoring neto coincide con el devuelto por get_sim_score
    # (no se combinan los distintos campos, simplemente se tiene en cuenta el
    #  campo fijado previamente con set_field_type):
    def get_net_score(self, q, d):
        return self.get_sim_score(q, d)

Probamos el _baseline scorer_ con una consulta y un par de documentos de ejemplo, para todos los campos posibles:

In [7]:
baseline_scorer = BaselineScorer(theIDF)

q = Query("stanford aoerc pool hours")
print("---------\n")
print("Consulta: ", q)
print(f"Vector de consulta:\n{baseline_scorer.get_query_vector(q)}")
print("\n---------\n")

d1 = query_dict[q]['http://events.stanford.edu/2014/February/18/']          # Ejemplo que tiene "body_hits".
d2 = query_dict[q]['https://cardinalrec.stanford.edu/facilities/aoerc/']    # Ejemplo que tiene "anchors".

for i,d in enumerate([d1,d2]):
    doc_vectors = baseline_scorer.get_doc_vector(q,d)
    print("Documento:\n", d)
    print(f"Vectores de documento:")
    similarities = {}
    for k in doc_vectors.keys(): # Para cada campo:
        baseline_scorer.set_field_type(k)
        similarity = baseline_scorer.get_sim_score(q,d)
        print(f"  {k} vector (computed similarity={similarity}):\n  {doc_vectors[k]}\n")
        similarities[k] = similarity
    # print(similarities)
    if i==0:
        assert similarities == {'url': 1, 'title': 1, 'headers': 6, 'anchors': 0, 'body_hits': 18}, \
          "Scorer de similaridad baseline utilizando pesos por defecto no obtiene resultado esperado para d1"
    elif i==1:
        assert similarities == {'url': 2, 'title': 0, 'headers': 0, 'anchors': 29, 'body_hits': 0}, \
          "Scorer de similaridad baseline utilizando pesos por defecto no obtiene resultado esperado para d2"
    print("---------\n")

print("Tests de clase BaselineScorer() superados.")

---------

Consulta:  stanford aoerc pool hours
Vector de consulta:
Counter({'stanford': 1, 'aoerc': 1, 'pool': 1, 'hours': 1})

---------

Documento:
 url: http://events.stanford.edu/2014/February/18/
 title: events at stanford tuesday february 18 2014
 headers: ['stanford university event calendar', 'teaching sex at stanford', 'rodin the complete stanford collection', 'stanford rec trx suspension training', 'memorial church open visiting hours', 'alternative transportation counseling tm 3 hour stanford univ shc employees retirees family members']
 body_hits: {'stanford': [239, 271, 318, 457, 615, 642, 663, 960, 966, 971], 'aoerc': [349, 401, 432, 530, 549, 578, 596], 'pool': [521]}
 body_length: 981
 pagerank: 1

Vectores de documento:
  url vector (computed similarity=1):
  Counter({'events': 1, 'stanford': 1, 'edu': 1, '2014': 1, 'february': 1, '18': 1})

  title vector (computed similarity=1):
  Counter({'events': 1, 'at': 1, 'stanford': 1, 'tuesday': 1, 'february': 1, '18': 1, '2